In [6]:
%matplotlib inline
import numpy as np
import scipy.stats as ss
from pandas import DataFrame

#Read all the files

Read the files in a directory (recursively).

In [7]:
import os

NEWLINE = '\n'
SKIP_FILES = set(['cmds'])

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path)
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    yield file_path, NEWLINE.join(lines).decode('cp1252', 'ignore')


#Make a dataframe of the spam.

In [2]:
def build_data_frame(path, classification):
    data_frame = DataFrame({'text': [], 'class': []})
    for file_name, text in read_files(path):
        data_frame = data_frame.append(
                DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
    return data_frame


In [4]:
HAM = 0
SPAM = 1

SOURCES = [
    ('data/spam',         SPAM),
    ('data/easy_ham',     HAM),
    ('data/hard_ham',     HAM),
    ('data/beck-s',       HAM),
    ('data/farmer-d',     HAM),
    ('data/kaminski-v',   HAM),
    ('data/kitchen-l',    HAM),
    ('data/lokay-m',      HAM),
    ('data/williams-w3',  HAM),
    ('data/BG',           SPAM),
    ('data/GP',           SPAM),
    ('data/SH',           SPAM)
    ]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
    data = data.append(build_data_frame(path, classification))
  
data = data.reindex(np.random.permutation(data.index))


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
# Fit learns the vocabulary.
# Transform does the count.
counts = count_vectorizer.fit_transform(np.asarray(data['text']))


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [7]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = np.asarray(data['class'])
classifier.fit(counts, targets)

NameError: name 'numpy' is not defined

In [8]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [9]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
  ('vectorizer',  CountVectorizer()),
  ('classifier',  MultinomialNB()) ])

pipeline.fit(np.asarray(data['text']), np.asarray(data['class']))
pipeline.predict(examples) # [1, 0]


NameError: name 'numpy' is not defined

In [10]:
from sklearn.cross_validation import KFold

k_fold = KFold(n=len(data), n_folds=6, indices=False)
scores = []
for train_indices, test_indices in k_fold:
  train_text = np.asarray(data[train_indices]['text'])
  train_y    = np.asarray(data[train_indices]['class'])

  test_text  = np.asarray(data[test_indices]['text'])
  test_y     = np.asarray(data[test_indices]['class'])

  pipeline.fit(train_text, train_y)
  score = pipeline.score(test_text, test_y)
  scores.append(score)

score = sum(scores) / len(scores)


/home/jeff/src/jma/talks/stats-intro/venv/lib/python3.4/site-packages/sklearn/cross_validation.py:69: DeprecationWarning: The indices parameter is deprecated and will be removed (assumed True) in 0.17
  stacklevel=1)


ValueError: Cannot have number of folds n_folds=6 greater than the number of samples: 0.

In [11]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier',         MultinomialNB()) ])


In [12]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('tfidf_transformer',  TfidfTransformer()),
  ('classifier',         MultinomialNB()) ])


NameError: name 'TfidfTransformer' is not defined